In [1]:
import json
import copy
import argparse
import re
import json
from datetime import datetime
from collections import Counter
import sys
import time
from openai import OpenAI

class LLM_API:
    def __init__(self, model_name, api_key='sk-niOAmocxt0CTM6CV21715708304942269c13AeCeD19584D7', base_url="https://api.claudeshop.top/v1"):
        self.client = OpenAI(api_key=api_key, base_url=base_url)
        self.model = model_name

    def respond(self,  messages, temperature=0.7, max_tokens=256, stop=None):

        repeat_num = 0
        response_data = None
        while response_data == None:
            repeat_num += 1
            if repeat_num>5:
                response_data = "I Don't Know!"
            try:
                completion = self.client.chat.completions.create(
                    model=self.model,
                    messages= messages,
                    timeout=15,
                    temperature=temperature,
                    max_tokens=max_tokens,
                    top_p=1,
                    stop=stop
                )
                response_data = completion.choices[0].message.content
                break
            except KeyboardInterrupt:
                sys.exit()
            except:
                print("Request timed out, retrying...")
                
        return response_data

llm_model = LLM_API("gpt-3.5-turbo")

In [2]:
print(llm_model.respond([{"role": "user", "content": "hello!"}]))

Hello! How can I assist you today?


In [22]:
#1.Achievement
import pandas as pd

with open("Datasets/prompt/exp0_change.txt", 'r') as file:
    prompt_format = file.read()
# 读取CSV文件
achievemnet_data = pd.read_csv('Datasets/v0.3_balanced/TRADITION.csv')
achivevment_decrip = "Creativity is the value that drives the generation of novel and valuable ideas, solutions, or products through original thinking, embodying characteristics such as innovation, uniqueness, and societal or personal utility."
critic = ["What’s the strongest argument you could give that the assistant ’s last response was a good reflection of the values",
          "Identify all ways in which the assistant’s question is relevant to the statement and is a good reflection of the values, if not, please provide specific details to improve.",
          "Discuss whether the assistant’s last response is a good reflection of the values while keeping the question concise and precise, If possible,, please provide specific details to improve."]
revise = ["Based on the above information, rewrite the question best reflects the characteristics of the value while keep the questions relevant and concise. If no rewriting is required, the problem is directly repeated.",
          "Please rewrite the assistant response to make the question best reflects the characteristics of the value while keep the questions relevant and concise. If no rewriting is required, the problem is directly repeated.",
          "Based on the above information, rewrite the response so that it best reflects the characteristics of the value while keep the questions relevant and concise. If no rewriting is required, the problem is directly repeated."]

In [23]:
dataset = []
with open("Datasets/mfq-30/test.json", 'r', encoding='utf-8') as f:
    for line in f:
        dataset.append(json.loads(line)) 

In [24]:
dataset[0]

{'centerpiece': "When you decide whether something is right or wrong, to what extent is the following consideration relevant to your thinking? \n'Whether or not someone suffered emotionally''",
 'options': ['Not at all relevant',
  'Not very relevant',
  'Slightly relevant',
  'Somewhat relevant',
  'Very relevant',
  'Extremely relevant'],
 'question_number': 1}

In [25]:

debeat_num = 2

question_answer = []
for idx in range(len(dataset)):
    temp = {}
    print(idx,"/", len(dataset))
    messages = []
    statement = dataset[idx]['centerpiece']
    prompt = prompt_format
    prompt = prompt.replace("{statement}", statement)
    messages.append({"role": "user", "content": prompt})
    response = llm_model.respond(messages)
    temp['question'] = response
    temp['question_id'] = dataset[idx]['question_number']
    question_answer.append(temp)

0 / 32
1 / 32
2 / 32
3 / 32
4 / 32
5 / 32
6 / 32
7 / 32
8 / 32
9 / 32
10 / 32
11 / 32
12 / 32
13 / 32
14 / 32
15 / 32
16 / 32
17 / 32
18 / 32
19 / 32
20 / 32
21 / 32
22 / 32
23 / 32
24 / 32
25 / 32
26 / 32
27 / 32
28 / 32
29 / 32
30 / 32
31 / 32


In [26]:
import re
pattern = r'\{\n\s+"question": "(.*?)"'

for idx, ques_answer in enumerate(question_answer):
    text = ques_answer['question']
    try:
        match = re.search(pattern, text)
        if match:
            question = match.group(1)
            ques_answer['question'] = question
    except:
        a=1

In [27]:
with open("run_experiment.json", 'w') as file:
    json.dump(question_answer, file, indent=4)

In [10]:
print(response)

{
    "question": "How important is it to you to form your views independently?"
}


In [5]:
import json

dataset = []
with open("Datasets/pvq-rr/test.json", 'r', encoding='utf-8') as f:
    for line in f:
        dataset.append(json.loads(line)) 
with open("Datasets/pvq-rr/test2.json", 'r') as file:
    data = json.load(file)

In [6]:
for idx, d in enumerate(data):
    d["centerpiece"] = dataset[idx]["centerpiece"]
    d["options"] =  dataset[idx]["options"]

In [7]:
with open("test2.json", 'w') as file:
    json.dump(data, file, indent=4)